In [ ]:
<div style="display: flex; justify-content: center;">
    <img src="../images/Logo-personal_Fondo_blanco.png" alt="Logo_personal" /> 
</div>
<p align="center">
  <h1 align="center">"credit card fraud predictive model with decision tree and random forest"<h1>
</p>

---

## Introduction
This project aims to generate a binomial logistic regression model to classify, based on an examination, whether cancer is benign or malignant.

The work is carried out based on analysis, compression, data cleaning, metrics, testing and validation of the model, with the following work path:

- Development
- Understanding the data
- Data cleaning
  - Check null values
- Correlation analysis
- Data standardization:
- Exploratory data analysis
- Model creation
  - Split training and test data
  - Create and train the model
  - Evaluate the accuracy of the model with metrics
  - Evaluate the model with cross validation
- Analysis of results
  - Get the probabilities
  - Get the coefficients
  - Confusion Matrix
- Test
  - Test eliminating the characteristics with lower coefficients
  - Test balancing the number of class records with SMOTE
- Regularizers
  - L1 Lasso
  - L2 Ridge
- Conclusions

## Data
The Data set ["Breast Cancer Wisconsin (Diagnostic)"](https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data) of kaggel contains Predict whether the cancer is benign or malignant. Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image.

**Attribute Information:**

* ID number
* Diagnosis (M = malignant, B = benign)
* radius (mean of distances from center to points on the perimeter)
* texture (standard deviation of gray-scale values)
* perimeter
* area
* smoothness (local variation in radius lengths)
* compactness (perimeter^2 / area - 1.0)
* concavity (severity of concave portions of the contour)
* concave points (number of concave portions of the contour)
* symmetry
* fractal dimension ("coastline approximation" - 1)

___